In [2]:
with open("./twitter_API.txt",'r') as token_file:
    contents = token_file.read().split('\n')
    consumer_key = contents[0]
    consumer_secret = contents[1]
    access_token = contents[2]
    access_token_secret = contents[3]
    
print(consumer_key,consumer_secret,access_token,access_token_secret,sep='\n')

qczJfz5mBRhWyCLwjICrXbaDe
vdgGqjZjfOagt00tKOCfgUFz7BlEgIQuhSM5bQqSLtfCmfZNxR
1055266392788873218-TEOrczbmpvnFb7PtWg56HveqtRBmcD
qQRInhGwvRdqfy9TlOJjOSyEqh41UhBjHWOwMROl5WzEi


In [3]:
import tweepy
# set the key and token
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)
search_term = 'AAPL'
## fill in your search query and store your results in a variable
results = api.search(q = search_term, lang = "en", result_type = "recent", count = 1000)

In [4]:
import tweepy
import time
import csv
import threading
lock = threading.Lock()

In [5]:
# the user id of the celebrities
# user_id_dict = {'Donald J. Trump':25073877,'Theresa May':747807250819981312,'Hillary Clinton':1339835893,
#                 'Barack Obama':813286,'WarrenBuffett':1364930179,'Nikki Haley':37666984,'George Soros':23125257,
#                'Melania Trump':818876014390603776,'Mike Pence':22203756}
user_id_dict = {'Donald J. Trump':25073877}
user_ids = []
for i in user_id_dict:
    user_ids.append(user_id_dict[i])
user_ids

[25073877]

In [7]:
# get the text of all tweets of those users

def get_tweets(user_ids):
    old_ids = []
    output = []
    lock.acquire()
    try:
        num = 0
        while len(user_ids) >= 1:
            try:
                user_id = user_ids[num]
                print('crawling user %s data...' % user_id)
                auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
                auth.set_access_token(access_token, access_token_secret)
                api = tweepy.API(auth)
                tweets = []
                new_tweets = api.user_timeline(user_id, count=200,tweet_mode='extended')
                tweets.extend(new_tweets)
                old = tweets[-1].id - 1
                while len(new_tweets) > 0:
                    new_tweets = api.user_timeline(user_id=user_id, count=200, max_id=old,tweet_mode='extended')
                    tweets.extend(new_tweets)
                    old = tweets[-1].id - 1
                    print('%s tweets downloaded' % (len(tweets)))
               
                # We output the id, text, time, favorits and retweets of the original tweet.
                out_tweets = [[tweet.id, tweet.created_at, tweet.full_text,tweet.retweeted, tweet.favorite_count,
                               tweet.retweet_count, tweet.user.id, tweet.user.name,
                               tweet.user.followers_count,tweet.user.created_at] for tweet in tweets]

                print(out_tweets)
                output.append(out_tweets)
                user_ids.remove(user_id)
                old_ids.append(user_id)
                
                # Write the result to a csv file.
                with open('./%s_tweets.csv' % (str(user_id)+'_'+tweets[0].user.name), 'w',encoding='utf-8') as file:
                    writer = csv.writer(file)
                    writer.writerows(out_tweets)
                print('saved data')
                
            except tweepy.TweepError as e:
                if e.reason=='Not authorized.':
                    print('this user not authorized.')
                    user_ids.remove(user_id)
                    old_ids.append(user_id)
                    continue
                else:print(e)
        #return output
    finally:
        lock.release()


In [8]:
# Get trump's tweet data and sve it to a csv
get_tweets([25073877])

crawling user 25073877 data...
400 tweets downloaded
600 tweets downloaded
799 tweets downloaded
997 tweets downloaded
1193 tweets downloaded
1393 tweets downloaded
1593 tweets downloaded
1793 tweets downloaded
1993 tweets downloaded
2193 tweets downloaded
2193 tweets downloaded
[[1071159669949911044, datetime.datetime(2018, 12, 7, 21, 49, 1), "It is being reported that Leakin' James Comey was told by Department of Justice attorneys not to answer the most important questions. Total bias and corruption at the highest levels of previous Administration. Force him to answer the questions under oath!", False, 18585, 5725, 25073877, 'Donald J. Trump', 56090661, datetime.datetime(2009, 3, 18, 13, 46, 38)], [1071146400019169280, datetime.datetime(2018, 12, 7, 20, 56, 17), 'Hopefully Mitch McConnell will ask for a VOTE on Criminal Justice Reform. It is extremely popular and has strong bipartisan support. It will also help a lot of people, save taxpayer dollars, and keep our communities safe. Go

In [70]:
# test version

# user_id = 25073877
# tweets = []
# new_tweets = api.user_timeline(user_id, count=200,tweet_mode='extended')
# tweets.extend(new_tweets)
# old = tweets[-1].id - 1
# while len(new_tweets) > 0:
#     new_tweets = api.user_timeline(user_id=user_id, count=200, max_id=old,tweet_mode='extended')
    
#     tweets.extend(new_tweets)
#     old = tweets[-1].id - 1
#     print('%s tweets downloaded' % (len(tweets)))

# tweets